### 3.17. (Tallant una barra d'acer)
Tenim una gran barra d'acer de longitud $n$, i volem tallar-la en trossos per a destinar-los a diferents usos. Cada tros de barra d'acer, en funció de la seva llargària, té un cost al mercat. Un tros de llargària $i$, amb $i \in \mathbb{Z}^+$ i $1 \leq i \leq n$, val $\mathbf{p_i}$ euros. Observeu que les llargàries dels trossos són unitats enteres. Per exemple, una figura de llargaria 4 té 8 maneres de dividir la barra. Sobre cadascun dels trossos s'indica el preu que se n'obté ($\mathbf{p_1 = 1}$, $\mathbf{p_2 = 5}$, $\mathbf{p_3 = 8}$, $\mathbf{p_4 = 9}$). La solució òptima és tallar la barra en dos trossos de longitud 2  que dóna un guany de 10€.

$(a)$ De quantes formes diferents es pot tallar una barra de llargària $n$?Raoneu la resposta.

$(b)$ Disganyeu un algorisme, el més eficient que pugueu, per a decidir com tallar una barra d'acer de longitud 1$n$ en trossos, de manera que es maximitzi el guany total que se n'obté. El vostre algorisme ha de dir quants talls s'han de fer en total, i2 a on.

*Nota:* Assumirem que fer un tall no indueix cap cost afegeix. Observeu que no es demana cap requisit sobre el nombre de talls a fer; podeu fer qualsevol nombre de talls entre $0$ i $n-1$.

#### Apartat a:
Per una barra de longitud $n$ en tenim $n-1$ punts de tall diferents. Per cada punt de tall tenim la decisió de si volem ``tallar`` o ``no tallar`` la barra d'acer. Per tant, el nombre de formes diferents que en tenim per una barra de llargària $n$ es de:

$$nº formes = \mathbf{2^{n-1}}$$

#### Apartat b:
##### Caracterització del problema
Per resoldre aquest exercici necesitem saber en cada moment, amb la forma que tenim actualment de la barra, quin es el preu que ens pagaran per ella. Per tant, l'objectiu es dividir la barra en totes les seves formes posibles, memoritzar el preu i després quedarnos amb la descomposició que te el màxim preu. Per fer la memoïtzació faré ús d'un array DP amb $n$ posicions que guardará quin es el preu màxim per cada barra de longitud desde 1 